## Getting Started with TensorBoard in Jupyter/Colab

**TensorBoard** is a powerful visualization tool provided with TensorFlow. It helps you understand, debug, and optimize your machine learning models by providing visualizations of the training process, model graphs, and other data.

The following cell loads the necessary **Jupyter/Colab extension** to run TensorBoard directly within the notebook environment.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Importing Libraries

We import **TensorFlow (as `tf`)** for building and training the model, and the standard **`datetime`** library to create unique log directories for each training run. This is a best practice for tracking different experiments.

In [ ]:
import tensorflow as tf
import datetime

## Clearing Previous Logs

This command ensures that any old log files from previous, potentially incomplete, runs are removed. This prevents TensorBoard from trying to combine data from unrelated experiments, which can cause confusion.

In [ ]:
# Clear any logs from previous runs
! rm -rf logs

## Loading and Preprocessing the MNIST Dataset

We'll use the **MNIST dataset**, the "Hello World" of deep learning, for digit classification. 

* **Loading Data:** The data is loaded into training and testing sets.
* **Normalization:** The pixel values (originally $0-255$) are scaled down to the range $0.0-1.0$ by dividing by $255.0$. This **normalization** is crucial for stable and faster model training.
* **Model Definition:** A simple **Sequential Keras model** is defined, consisting of:
    * **`Flatten`**: Converts the $28 \times 28$ image into a flat $784$-element vector.
    * **`Dense(512, activation='relu')`**: A hidden layer with 512 neurons and the **ReLU** activation function.
    * **`Dropout(0.2)`**: Randomly sets 20% of the input units to 0 at each update during training, which is a powerful technique for preventing **overfitting**.
    * **`Dense(10, activation='softmax')`**: The output layer with 10 neurons (for 10 classes) and the **softmax** activation, which provides probability scores for each class.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

## Configuring and Training the Model with the TensorBoard Callback

This is the core section where we integrate TensorBoard into the Keras training process.

* **Model Compilation:** The model is compiled with the **`adam`** optimizer and the **`sparse_categorical_crossentropy`** loss function (suitable for integer-labeled classification).
* **Log Directory Creation:** A unique log directory path is created using the current timestamp. This allows running the training multiple times without overwriting previous results, making experiment comparison easy.
* **`tf.keras.callbacks.TensorBoard`:** This **callback** is the bridge between the Keras training loop and TensorBoard. The key argument is:
    * **`log_dir=log_dir`**: Specifies where to write the log files.
    * **`histogram_freq=1`**: Tells TensorBoard to compute and log histogram data (weights and biases distributions) for the layers every epoch. This is highly valuable for debugging and seeing how weights change over time.
* **`model.fit()`:** The training process begins. We pass the **`tensorboard_callback`** in the `callbacks` list, which enables the logging of metrics (loss, accuracy) and histograms.

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

## Launching TensorBoard

This **magic command** starts the TensorBoard server and embeds the visualization interface directly into the output of this cell. It will display the logged metrics and histograms, allowing you to analyze the training performance. The `--logdir logs/fit` argument tells TensorBoard to look in the main log folder, where all our timestamped experiment folders reside.

In [ ]:
%tensorboard --logdir logs/fit